In [1]:
from test_eval import *
from tqdm import tqdm

In [2]:
def load_idxs(path='ids.txt'):
    tests = []
    with open(path, "r") as f:
        for line in f:
            line = line.split("\t")
            test = [int(i) for i in line]
            tests.append(test)
    return tests

def gen_random_idxs(doc_ids, times=1000, dump_to='ids.txt'):
    import random
    with open(dump_to, "w") as f:
        while times > 0:
            test_docs = random.choices(doc_ids, k=len(doc_ids))
            output = "\t".join([str(i) for i in test_docs]) + "\n"
            f.write(output)
            times -= 1

In [3]:
def load_results(result_root):
    training_sets = os.listdir(result_root)
    sd_abs, sd_ext, sd_mix = merge_results(result_root, training_sets, True)
    return sd_abs, sd_ext, sd_mix

In [4]:
# Configurations 
result_root = "../../exp/result_bert_base_uncased_5_15/15"
level="summary"
method = spearmanr
sd_abs, sd_ext, sd_mix = load_results(result_root)

mlist = utils.get_metrics_list(sd_mix)
print(mlist)

if not os.path.exists("ids.txt"):
    gen_random_idxs(list(sd_mix.keys()))

tests = load_idxs()

['B_big_patent_pref_ordered', 'B_billsum_pref_ordered', 'B_cnn_dailymail_pref_ordered', 'B_newsroom_pref_ordered', 'B_scientific_papers_pref_ordered', 'bert_f_score', 'bert_precision_score', 'bert_recall_score', 'js-2', 'litepyramid_recall', 'metric_bert_score_f1', 'metric_bert_score_precision', 'metric_bert_score_recall', 'metric_blanc', 'metric_blanc_tune', 'metric_bleu', 'metric_cider', 'metric_meteor', 'metric_mover_score', 'metric_s3_pyr', 'metric_s3_resp', 'metric_summaqa_avg_fscore', 'metric_summaqa_avg_prob', 'metric_supert', 'mover_score', 'rouge_1_f_score', 'rouge_1_precision', 'rouge_1_recall', 'rouge_2_f_score', 'rouge_2_precision', 'rouge_2_recall', 'rouge_l_f_score', 'rouge_l_precision', 'rouge_l_recall']


In [5]:
# Test hyp H_0 corrA >= corrB, alternative hyp H_a: corrA < corrB

targetA = ['B_cnn_dailymail_pref_ordered', 'B_billsum_pref_ordered', 'B_big_patent_pref_ordered',  'B_scientific_papers_pref_ordered']
targetB = ['metric_blanc_tune', 'metric_supert', 'metric_summaqa_avg_fscore', 'metric_summaqa_avg_prob']

In [6]:
def sig_test(sd):
    f = open("sig.tsv", "w", encoding="utf-8")
    for sysA in targetA:
        f.write(sysA + "\t")
        for sysB in targetB:
            print(sysA, sysB)
            corrA = calc_corr(level, method, ('litepyramid_recall', sysA), sd, None)
            corrB = calc_corr(level, method, ('litepyramid_recall', sysB), sd, None)
            if corrA < corrB: 
                print("Skip")
                f.write("-\t")
                continue
            
            count = 0
            for test in tqdm(tests):
                corrA = calc_corr(level, method, ('litepyramid_recall', sysA), sd, None, test)
                corrB = calc_corr(level, method, ('litepyramid_recall', sysB), sd, None, test)
                if corrA < corrB: count += 1
            pvalue = count / len(tests)
            print("P-value:", pvalue)
            f.write("%.2f\t" % pvalue)
        f.write("\n")
    f.write("\t".join([""] + targetB))
    f.close()

In [7]:
sig_test(sd_ext)

B_cnn_dailymail_pref_ordered metric_blanc_tune


100%|██████████| 1000/1000 [02:33<00:00,  6.53it/s]


P-value: 0.491
B_cnn_dailymail_pref_ordered metric_supert


100%|██████████| 1000/1000 [02:31<00:00,  6.60it/s]


P-value: 0.081
B_cnn_dailymail_pref_ordered metric_summaqa_avg_fscore


...

P-value: 0.293
B_scientific_papers_pref_ordered metric_summaqa_avg_fscore


100%|██████████| 1000/1000 [02:29<00:00,  6.71it/s]


P-value: 0.41
B_scientific_papers_pref_ordered metric_summaqa_avg_prob


100%|██████████| 1000/1000 [02:28<00:00,  6.72it/s]

P-value: 0.172
